## Supervised models
This notebook is intended for giving an introduction the ML supervised models that can be used for Covid detection.

For this notebook to find the new modules created for this project, we need to set its path to be in the root directory.

In [1]:
import sys
sys.path.append("../")

## Loading packages and dependencies

In [2]:
import numpy as np

from src.features.extract_features import load_extracted_features
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from src.models.build_model import train_advanced_supervised_model, evaluate_model


# Path to the raw data and preprocessed data
raw_data_dir = '../data/raw/COVID-19_Radiography_Dataset/'
IMG_SIZE = 299  # Resize images to IMG_SIZExIMG_SIZE pixels

## Extracting features from images

In [3]:
F_healthy, y_healthy, X_healthy  = load_extracted_features(images_dir=raw_data_dir+'{}/images',
                                                       category='NORMAL', dataset_label=0, random_seed=42, samples=781, 
                                                       image_size=IMG_SIZE, image_resized=True, augmentor=True)                                                       
F_sick, y_sick, X_sick = load_extracted_features(images_dir=raw_data_dir+'{}/images',
                                                        category=['COVID','Viral Pneumonia','Lung_Opacity'], dataset_label=1,
                                                        image_size=IMG_SIZE, image_resized=True)


Loaded images for NORMAL: 10973 resized images, 10973 features, and 10973 labels.
Loaded images for ['COVID', 'Viral Pneumonia', 'Lung_Opacity']: 10973 resized images, 10973 features, and 10973 labels.


## Normalizing features

In [4]:
# Combine datasets
# Keep 7 features only
# F_healthy = [f[:7] for f in F_healthy]
# F_sick = [f[:7] for f in F_sick]

F = np.vstack((F_healthy, F_sick)) #image features
X = np.vstack((X_healthy, X_sick)) #image data
y = np.concatenate((y_healthy, y_sick)) #labels

X_img_train, X_img_test, X_feat_train, X_feat_test, y_train, y_test = train_test_split(
    X, F, y, test_size=0.2, random_state=42, stratify=y
)
print(f"X_train shape: {X_feat_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_feat_test.shape}, y_test shape: {y_test.shape}")
print(f"X_img_train shape: {X_img_train.shape}")

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_feat_train)
X_test = scaler.transform(X_feat_test)
X_feat_train = scaler.fit_transform(X_feat_train)
X_feat_test = scaler.transform(X_feat_test)

X_train shape: (17556, 14), y_train shape: (17556,)
X_test shape: (4390, 14), y_test shape: (4390,)
X_img_train shape: (17556, 299, 299)


In [5]:
# Reshape image data for CNN
X_img_train = X_img_train.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_img_test = X_img_test.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

print(f"X_img_train shape: {X_img_train.shape}")
print(f"X_img_test shape: {X_img_test.shape}")

X_img_train shape: (17556, 299, 299, 1)
X_img_test shape: (4390, 299, 299, 1)


## Training and evaluating models

### Convolutional Neural Networks (CNN)

✅ Strengths:
* Highly accurate for image tasks.
* Learns complex patterns automatically.
* Works well with large image datasets.

❌ Weaknesses:
* Computationally expensive (needs GPUs).
* Requires large labeled datasets.
* Not easily interpretable.

In [6]:
model, history = train_advanced_supervised_model([X_img_train, X_feat_train], y_train, [X_img_test, X_feat_test], y_test, 
                                [IMG_SIZE,X_feat_train.shape[1]], 10)

2025-02-16 07:04:35.417143: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2025-02-16 07:04:35.417348: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 128.00 GB
2025-02-16 07:04:35.417358: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 48.00 GB
2025-02-16 07:04:35.417574: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-16 07:04:35.417588: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 299, 299,  │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 149, 149,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 149, 149,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 74, 74,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 74, 74,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 37, 37,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │        480 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 175232)    │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 175264)    │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │ 11,216,960 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         65 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,310,177 (43.14 MB)

 Trainable params: 11,310,177 (43.14 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


2025-02-16 07:04:39.362688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1098/1098 ━━━━━━━━━━━━━━━━━━━━ 41s 36ms/step - accuracy: 0.6837 - loss: 0.7658 - val_accuracy: 0.8157 - val_loss: 0.4239
Epoch 2/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.7988 - loss: 0.4932 - val_accuracy: 0.8280 - val_loss: 0.3908
Epoch 3/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.8320 - loss: 0.4017 - val_accuracy: 0.8335 - val_loss: 0.4165
Epoch 4/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.8391 - loss: 0.4202 - val_accuracy: 0.8312 - val_loss: 0.4703
Epoch 5/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.8120 - loss: 0.7277 - val_accuracy: 0.8301 - val_loss: 0.5816
Epoch 6/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.7938 - loss: 1.9612 - val_accuracy: 0.7788 - val_loss: 6.5278
Epoch 7/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.7963 - loss: 5.9550 - val_accuracy: 0.7879 - val_loss: 18.0324
Epoch 8/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 39s 35ms/step - accuracy: 0.7703 - loss: 33.

In [7]:
train_loss, train_acc = history.history['loss'][-1], history.history['accuracy'][-1]
print(f"Train Accuracy: {train_acc:.4f}, Train Loss: {train_loss:.4f}")

test_loss, test_acc = evaluate_model(model, [X_img_test, X_feat_test], y_test, model_type="CNN")
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")


Train Accuracy: 0.7573, Train Loss: 552.9791
138/138 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7843 - loss: 1202.9209
Test Accuracy: 0.7802, Test Loss: 1256.7651


### Capsule Network

✅ Strengths:

* CapsNet learns spatial relationships better than CNNs.
* More robust to rotation & deformation in medical images.
* Less training data required but computationally expensive.

#### RBF kernel

In [8]:
model, history = train_advanced_supervised_model([X_img_train, X_feat_train], y_train, [X_img_test, X_feat_test], y_test, 
                                [IMG_SIZE,X_feat_train.shape[1]], 10, model_type="Capsule Network")

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 299, 299,  │        640 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 299, 299,  │     18,464 │ conv2d_3[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 2860832)   │          0 │ conv2d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 32, 89401) │          0 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32, 128)   │ 11,443,456 │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 32)        │        480 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 4096)      │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 32)        │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 4128)      │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 64)        │    264,256 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 64)        │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1)         │         65 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,727,361 (44.74 MB)

 Trainable params: 11,727,361 (44.74 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 81s 73ms/step - accuracy: 0.6714 - loss: 0.6576 - val_accuracy: 0.7670 - val_loss: 0.5639
Epoch 2/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 76s 70ms/step - accuracy: 0.6841 - loss: 1.7392 - val_accuracy: 0.7567 - val_loss: 4.2633
Epoch 3/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 72s 65ms/step - accuracy: 0.6663 - loss: 30.0817 - val_accuracy: 0.7631 - val_loss: 50.1503
Epoch 4/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 69s 63ms/step - accuracy: 0.6627 - loss: 220.3275 - val_accuracy: 0.7770 - val_loss: 174.5305
Epoch 5/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 68s 62ms/step - accuracy: 0.6666 - loss: 704.1218 - val_accuracy: 0.7583 - val_loss: 626.5435
Epoch 6/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 67s 61ms/step - accuracy: 0.6619 - loss: 1682.8470 - val_accuracy: 0.7474 - val_loss: 913.6430
Epoch 7/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 69s 63ms/step - accuracy: 0.6691 - loss: 1920.3698 - val_accuracy: 0.7679 - val_loss: 680.5044
Epoch 8/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 69s 63ms/step 

In [9]:
train_loss, train_acc = history.history['loss'][-1], history.history['accuracy'][-1]
print(f"Train Accuracy: {train_acc:.4f}, Train Loss: {train_loss:.4f}")

test_loss, test_acc = evaluate_model(model, [X_img_test, X_feat_test], y_test, model_type="Capsule Network")
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

Train Accuracy: 0.6682, Train Loss: 221.8877
138/138 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5457 - loss: 59.2272
Test Accuracy: 0.5515, Test Loss: 58.8980


### Transfer learning

✅ Strengths
* Transfer learning reduces training time while maintaining high accuracy.
* Fine-tuning improves performance when sufficient data is available.
* Combining deep features with statistical features can enhance results.

In [10]:
model, history = train_advanced_supervised_model([X_img_train, X_feat_train], y_train, [X_img_test, X_feat_test], y_test, 
                                [IMG_SIZE,X_feat_train.shape[1]], 10, model_type="Transfer Learning")

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 299, 299,  │          0 │ input_layer_4[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 299, 299,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 299, 299,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 301, 301,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 150, 150,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 150, 150,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 150, 150,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 150, 150,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 150, 150,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 150, 150,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 150, 150,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 150, 150,  │        512 │ block1a_se_excit

 Total params: 4,224,388 (16.11 MB)

 Trainable params: 174,817 (682.88 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 75s 64ms/step - accuracy: 0.5676 - loss: 0.7188 - val_accuracy: 0.6592 - val_loss: 0.6343
Epoch 2/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 71s 65ms/step - accuracy: 0.6252 - loss: 0.6682 - val_accuracy: 0.6722 - val_loss: 0.6263
Epoch 3/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 68s 62ms/step - accuracy: 0.6386 - loss: 0.6581 - val_accuracy: 0.6843 - val_loss: 0.6220
Epoch 4/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 67s 61ms/step - accuracy: 0.6443 - loss: 0.6636 - val_accuracy: 0.6699 - val_loss: 0.6214
Epoch 5/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 68s 62ms/step - accuracy: 0.6378 - loss: 0.6746 - val_accuracy: 0.6811 - val_loss: 0.6205
Epoch 6/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 68s 62ms/step - accuracy: 0.6435 - loss: 0.6755 - val_accuracy: 0.6829 - val_loss: 0.6205
Epoch 7/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 73s 67ms/step - accuracy: 0.6329 - loss: 0.7013 - val_accuracy: 0.6909 - val_loss: 0.6224
Epoch 8/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 70s 64ms/step - accuracy: 0.6332 -

In [11]:
train_loss, train_acc = history.history['loss'][-1], history.history['accuracy'][-1]
print(f"Train Accuracy: {train_acc:.4f}, Train Loss: {train_loss:.4f}")

test_loss, test_acc = evaluate_model(model, [X_img_test, X_feat_test], y_test, model_type="Transfer Learning")
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

Train Accuracy: 0.6233, Train Loss: 0.7245
138/138 ━━━━━━━━━━━━━━━━━━━━ 13s 92ms/step - accuracy: 0.6774 - loss: 0.6268
Test Accuracy: 0.6781, Test Loss: 0.6213
